# BREAKFAST AUDIENCE MODEL DEVELOPMENT

In [1]:
!pip install plotly --user

In [2]:
# user inputs
project_id = 'cdp-ml-prd-ba23'
bucket_for_model = 'cdp-ml-prd-ai-ml'
folder_path = 'artifacts/breakfast_audience'
checkpoint_table = 'cdp_audiences.t_mdl_breakfast_audience_checkpoints'
max_vif_allowed = 2
max_pvalue_allowed = 0.05

In [3]:
# import packages
import numpy as np
import pandas as pd
from google.cloud import bigquery, storage
import statsmodels.api as sm
from statsmodels.formula.api import logit
from sklearn import metrics
import plotly.express as px
from statsmodels.stats.outliers_influence import variance_inflation_factor
from datetime import datetime as dt

## READ MODEL DATA

In [4]:
this_query = \
'''
SELECT
  UUID,
  SF_CONTACT_ID,
  IFNULL(LVL_UP_NUM_VISITS, 0) LVL_UP_NUM_VISITS,
  IFNULL(LVL_UP_LOY_BAL, 0) LVL_UP_LOY_BAL, 
  IFNULL(LVL_UP_LOY_TOT_PTS, 0) LVL_UP_LOY_TOT_PTS,
  DAYS_SINCE_LAST_VISIT,
  IFNULL(AVG_CHECK_BKFST, 0) AVG_CHECK_BKFST,
  IFNULL(AVG_CHECK_LUNCH, 0) AVG_CHECK_LUNCH,
  IFNULL(AVG_CHECK_AFTN, 0) AVG_CHECK_AFTN,
  IFNULL(AVG_CHECK_DINNER, 0) AVG_CHECK_DINNER,
  IFNULL(AVG_CHECK_PM_SNK, 0) AVG_CHECK_PM_SNK,
  IFNULL(AVG_CHECK_LATE_NT, 0) AVG_CHECK_LATE_NT,
  FREQ_BKFST,
  FREQ_LUNCH,
  FREQ_AFTN,
  FREQ_DINNER,
  FREQ_PM_SNK,
  FREQ_LATE_NT,
  FREQ_1MO,
  IFNULL(AVG_CHECK_1MO, 0) AVG_CHECK_1MO,
  FREQ_3MO,
  IFNULL(AVG_CHECK_3MO, 0) AVG_CHECK_3MO,
  FREQ_6MO,
  IFNULL(AVG_CHECK_6MO, 0) AVG_CHECK_6MO,
  FREQ_12MO,
  IFNULL(AVG_CHECK_12MO, 0) AVG_CHECK_12MO,
  FREQ_MARCH,
  IFNULL(PCT_DIGITAL_SPEND, 0) PCT_DIGITAL_SPEND,
  IFNULL(PCT_DISCOUNTED, 0) PCT_DISCOUNTED,
  CLTV_12MO,
  ALIVE_PROB,
  FUTURE_12MO_PURCHASE,
   (CASE
      WHEN FREQ_BKFST = 0 THEN 0
    ELSE
    1
  END
    ) AS CUSTOMER_TYPE
FROM
  `cdp-prd-6370.gold.t_fact_unified_customer`
WHERE
  FREQ_CATEGORY_BIN <> 'Inactive'
  ;
'''

In [5]:
# read model data
client = bigquery.Client(project_id)
job = client.query(this_query)
model_data = job.to_dataframe()

## FEATURE SELECTION

In [6]:
# profile of CUSTOMER_TYPE 0: NON-BREAKFAST, 1:BREAKFAST
# dependent var
dependent_variable = 'CUSTOMER_TYPE'
dist_dependent_variable = model_data[dependent_variable].value_counts()
dist_dependent_variable/dist_dependent_variable.sum()

0    0.797189
1    0.202811
Name: CUSTOMER_TYPE, dtype: float64

In [7]:
# feature list
feature_list = ['LVL_UP_NUM_VISITS',
                'LVL_UP_LOY_BAL',
                'DAYS_SINCE_LAST_VISIT', 
                # 'AVG_CHECK_BKFST', # this is similar to the dependent variable, hence excluded
                'AVG_CHECK_LUNCH', 
                'AVG_CHECK_AFTN', 
                'AVG_CHECK_DINNER',
                'AVG_CHECK_PM_SNK', 
                'AVG_CHECK_LATE_NT', 
                # 'FREQ_BKFST', # this is similar to the dependent variable, hence excluded
                'FREQ_LUNCH',
                'FREQ_AFTN', 
                'FREQ_DINNER', 
                'FREQ_PM_SNK', 
                'FREQ_LATE_NT', 
                'FREQ_1MO',
                'AVG_CHECK_1MO', 
                'FREQ_3MO', 
                'AVG_CHECK_3MO', 
                'FREQ_6MO',
                'AVG_CHECK_6MO', 
                'FREQ_12MO', 
                'AVG_CHECK_12MO', 
                'FREQ_MARCH',
                'PCT_DIGITAL_SPEND', 
                'PCT_DISCOUNTED',
                'CLTV_12MO', 
                'ALIVE_PROB', 
                'FUTURE_12MO_PURCHASE'
               ]
# feature drop list
drop_list = []

In [8]:
# multi-collinearity check and variable selection
max_vif_found = 1000
while max_vif_found > max_vif_allowed:
    correlation_matrix = model_data[feature_list].corr()
    w, v = np.linalg.eig(correlation_matrix)
    # list out linearly dependent variables
    for i in range(len(w)):
        if np.abs(w[i]) < 1:
            linearly_dependent = np.array(feature_list)[np.abs(v[:,0]) > 0.20]
            vif_values = [variance_inflation_factor(model_data[linearly_dependent], i) for i in range(len(linearly_dependent))]
            max_vif_found = np.max(vif_values)
            if max_vif_found > max_vif_allowed:
                max_vif_position = np.argmax(vif_values)
                drop_list.append(linearly_dependent[max_vif_position])
                break
    print('DROP LIST: ' + ', '.join(drop_list))
    feature_list = [x for x in feature_list if x not in drop_list]

DROP LIST: FREQ_12MO
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO, LVL_UP_LOY_BAL
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO, LVL_UP_LOY_BAL, ALIVE_PROB
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO, LVL_UP_LOY_BAL, ALIVE_PROB, LVL_UP_NUM_VISITS
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO, LVL_UP_LOY_BAL, ALIVE_PROB, LVL_UP_NUM_VISITS, CLTV_12MO
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO, LVL_UP_LOY_BAL, ALIVE_PROB, LVL_UP_NUM_VISITS, CLTV_12MO, AVG_CHECK_3MO
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO, LVL_UP_LOY_BAL, ALIVE_PROB, LVL_UP_NUM_VISITS, CLTV_12MO, AVG_CHECK_3MO, FREQ_1MO
DROP LIST: FREQ_12MO, FUTURE_12MO_PURCHASE, FREQ_6MO, FREQ_3MO, LVL_UP_LOY_BAL, ALIVE_PROB, LVL_UP_NUM_VISITS, CLTV_12MO, AVG_CHECK_3MO, 

In [9]:
# variables to work with
feature_list

['DAYS_SINCE_LAST_VISIT',
 'AVG_CHECK_LUNCH',
 'AVG_CHECK_AFTN',
 'AVG_CHECK_DINNER',
 'AVG_CHECK_PM_SNK',
 'AVG_CHECK_LATE_NT',
 'FREQ_LUNCH',
 'FREQ_AFTN',
 'FREQ_DINNER',
 'FREQ_PM_SNK',
 'FREQ_LATE_NT',
 'AVG_CHECK_1MO',
 'AVG_CHECK_12MO',
 'FREQ_MARCH',
 'PCT_DIGITAL_SPEND',
 'PCT_DISCOUNTED']

In [10]:
# correlation among the variables
corr_matrix = model_data[feature_list].corr()
corr_matrix

,DAYS_SINCE_LAST_VISIT,AVG_CHECK_LUNCH,AVG_CHECK_AFTN,AVG_CHECK_DINNER,AVG_CHECK_PM_SNK,AVG_CHECK_LATE_NT,FREQ_LUNCH,FREQ_AFTN,FREQ_DINNER,FREQ_PM_SNK,FREQ_LATE_NT,AVG_CHECK_1MO,AVG_CHECK_12MO,FREQ_MARCH,PCT_DIGITAL_SPEND,PCT_DISCOUNTED
DAYS_SINCE_LAST_VISIT,1.000000,-0.131785,-0.152235,-0.138565,-0.139210,-0.090641,-0.233339,-0.226417,-0.246194,-0.189542,-0.121844,-0.374903,0.002142,-0.350013,-0.129371,-0.320793
AVG_CHECK_LUNCH,-0.131785,1.000000,0.077866,0.025153,0.024594,-0.022860,0.219380,0.065817,0.064142,0.029288,-0.006387,0.127575,0.322654,0.064024,0.060633,-0.041343
AVG_CHECK_AFTN,-0.152235,0.077866,1.000000,0.101891,0.087891,0.038467,0.076242,0.294745,0.131148,0.083423,0.037940,0.132340,0.267450,0.085965,0.035363,-0.025285
AVG_CHECK_DINNER,-0.138565,0.025153,0.101891,1.000000,0.125699,0.018004,0.026460,0.052910,0.317841,0.083192,0.014425,0.148026,0.468825,0.055402,0.098263,-0.045742
AVG_CHECK_PM_SNK,-0.139210,0.024594,0.087891,0.125699,1.000000,0.153881,0.042170,0.087488,0.154252,0.431734,0.116421,0.137314,0.231592,0.090855,0.047060,-0.012047
AVG_CHECK_LATE_NT,-0.090641,-0.022860,0.038467,0.018004,0.153881,1.000000,0.007068,0.061763,0.065813,0.180506,0.409877,0.087492,0.129586,0.066611,0.037010,-0.004068
FREQ_LUNCH,-0.233339,0.219380,0.076242,0.026460,0.042170,0.007068,1.000000,0.372777,0.263612,0.156576,0.062180,0.125449,-0.091730,0.439750,-0.002400,0.028197
FREQ_AFTN,-0.226417,0.065817,0.294745,0.052910,0.087488,0.061763,0.372777,1.000000,0.406719,0.270288,0.148684,0.119464,-0.087265,0.420574,-0.019512,0.034411
FREQ_DINNER,-0.246194,0.064142,0.131148,0.317841,0.154252,0.065813,0.263612,0.406719,1.000000,0.396317,0.151779,0.166252,-0.004549,0.384625,0.016212,0.024819
FREQ_PM_SNK,-0.189542,0.029288,0.083423,0.083192,0.431734,0.180506,0.156576,0.270288,0.396317,1.000000,0.344622,0.125371,-0.026294,0.299947,0.008459,0.027552


## MODEL BUILDING

In [ ]:
# sign rules 
# some variables are expected to have positive impact (odds ratio > 1)
# some other variables are expected to have a negative impact (odds ratio < 1)
# rest can have anything
# if there is a violation of sign rule, those variables are dropped at the beginning
positive_variables = ['PCT_DISCOUNTED']
negative_variables = ['AVG_CHECK_LATE_NT']
this_formula = dependent_variable + ' ~ ' + ' + '.join(feature_list)
propensity_model = logit(this_formula, model_data[feature_list + [dependent_variable]]).fit()
drop_list = []
for x in positive_variables:
    if propensity_model.params[propensity_model.params.index == x][0] < 0:
        print('SIGN RULE VIOLATED: ' + x + ' REASON: EXPECTED POSITIVE - WILL BE DROPPED')
        drop_list.append(x)
for x in negative_variables:
    if propensity_model.params[propensity_model.params.index == x][0] > 0:
        print('SIGN RULE VIOLATED: ' + x + ' REASON: EXPECTED NEGATIVE - WILL BE DROPPED')
        drop_list.append(x)
feature_list = [x for x in feature_list if x not in drop_list]

In [ ]:
# dropping insignificant variables
max_pvalue = 1
while max_pvalue > max_pvalue_allowed:
    this_formula = dependent_variable + ' ~ ' + ' + '.join(feature_list)
    propensity_model = logit(this_formula, model_data[feature_list + [dependent_variable]]).fit()
    max_pvalue = np.max(propensity_model.pvalues[1:])
    max_pvalue_index = np.argmax(propensity_model.pvalues[1:])
    if max_pvalue > max_pvalue_allowed:
        variable_with_max_pvalue = feature_list[max_pvalue_index]
        print('DROPPED: ' + variable_with_max_pvalue + ' P VALUE: ' + str(max_pvalue))
        feature_list = [x for x in feature_list if x != variable_with_max_pvalue]    

In [ ]:
# model summary
propensity_model.summary()

In [ ]:
# odds ratio
np.exp(propensity_model.params)

In [ ]:
# confidence interval for odds ratio
np.exp(propensity_model.conf_int(alpha=0.05))

In [ ]:
# ROC
predicted_probabilities = propensity_model.predict()
auc = np.round(metrics.roc_auc_score(model_data[dependent_variable], predicted_probabilities), 2)
print('AUC: ' + str(auc))

## CHECKING MODEL GENERALIZATION THROUGH TRAIN TEST SPLIT

In [ ]:
# generate random numbers
s = np.random.uniform(0,1,model_data.shape[0])
training_data = model_data[s < 0.8]
test_data = model_data[s >= 0.8]
train_model = logit(this_formula, training_data[feature_list + [dependent_variable]]).fit()
train_auc = np.round(metrics.roc_auc_score(training_data[dependent_variable], 
                                           train_model.predict()
                                          ), 2
                    )
test_auc = np.round(metrics.roc_auc_score(test_data[dependent_variable], 
                                          train_model.predict(test_data)
                                         ), 2
                   )
print('TRAIN AUC: ' + str(train_auc) + '\nTEST AUC: ' + str(test_auc))

## DECILES CREATION

In [ ]:
# deciles
deciles_data = model_data[['UUID', 'SF_CONTACT_ID', dependent_variable]]
deciles_data['prob'] = predicted_probabilities
deciles_data['decile'] = pd.qcut(deciles_data['prob'], 10, labels = False)
deciles_pivot = deciles_data.pivot_table(index='decile', columns=dependent_variable, values='UUID', aggfunc=np.size)
deciles_pivot.columns = ['NON-BREAKFAST', 'BREAKFAST']

In [ ]:
fig = px.bar(deciles_pivot, 
             x=deciles_pivot.index + 1, 
             y=['BREAKFAST', 'NON-BREAKFAST'], 
             title="DECILE COMPOSITION",
             labels={"value": "COUNT", "x": "DECILE"}
            )
fig.update_layout(barmode='relative')
fig.show()

## SAVING MODEL

In [ ]:
# reading checkpoints table
save_this_model = False
if save_this_model:
    this_query = \
    '''
    SELECT * FROM 
    ''' + project_id + '.' + checkpoint_table
    bq_client = bigquery.Client(project_id)
    job = client.query(this_query)
    checkpoint_data = job.to_dataframe()

    # creating checkpoint inputs
    checkpoint_data_size = checkpoint_data.shape[0]
    if checkpoint_data_size > 0:
        model_version = int(np.max(checkpoint_data['MODEL_VERSION']) + 1)
    else:
        model_version = 1
    model_path = '/gcs/' + bucket_for_model + '/' + folder_path + '/model_' + str(model_version) + '.pkl'
    model_data_size = model_data.shape[0]
    created_tms = dt.now().strftime('%Y-%m-%d %H:%M:%S')

    # saving the model parameters in GCS
    propensity_model.params.to_pickle('breakfast_audience_parameters.pkl')
    client = storage.Client(project = project_id)
    bucket = client.get_bucket(bucket_for_model)
    blob = bucket.blob(folder_path + '/model_' + str(model_version) + '.pkl')
    blob.upload_from_filename('breakfast_audience_parameters.pkl')

    # inserting row into checkpoint table
    rows_to_insert = [{u"MODEL_VERSION": model_version, 
                       u"MODEL_PATH": model_path, 
                       u"TRAINING_DATASET_SIZE": model_data_size,
                       u"AUC": auc,
                       u"CREATED_TMS": created_tms
                      }]
    table = bq_client.get_table(project_id + '.' + checkpoint_table)
    errors = bq_client.insert_rows_json(table, rows_to_insert)
    if errors == []:
        print("success")